In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: marcodia
"""
import numpy as np
import random
import xarray as xr
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
import tensorflow as tf
import import_ipynb
import sys
import os 

import network_arch as network
import metrics
import plot
import settings 

importing Jupyter notebook from network_arch.ipynb
importing Jupyter notebook from metrics.ipynb
importing Jupyter notebook from plot.ipynb
importing Jupyter notebook from settings.ipynb


In [8]:
#%% >>>>> ADDITIONAL FUNCTIONS >>>>>
def is_ndjf(month):
    return np.logical_or(month<=2, month>=11)

def is_ndjfm(month):
    return np.logical_or(month<=3, month>=11)

# MAKE THE NN ARCHITECTURE
def make_model():
    # Define and train the model
    tf.keras.backend.clear_session()
    model = network.defineNN(HIDDENS,
                             input1_shape = X_train.shape[1],
                             output_shape=NLABEL,
                             ridge_penalty1=RIDGE1,
                             dropout=DROPOUT,
                             act_fun='relu',
                             network_seed=NETWORK_SEED)
    
    loss_function = tf.keras.losses.CategoricalCrossentropy()    
    model.compile(
                  optimizer = tf.keras.optimizers.Adam(learning_rate=LR_INIT),
                  loss = loss_function,
                  metrics = [
                      tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None),
                      metrics.PredictionAccuracy(NLABEL)
                      ]
                  )           
    return model, loss_function

#---------------------------------------------------
#LEARNING RATE CALLBACK FUNCTION
def scheduler(epoch, lr):
    # This function keeps the initial learning rate for the first ten epochs
    # and decreases it exponentially after that.
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [12]:
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from tensorflow.keras import regularizers
import json

In [3]:
#Need to pre-process data as done in NeuralNetwork_train.ipynb

In [4]:
#Code adopted from Keras online tutorial: https://keras.io/keras_tuner/ 

In [28]:
#Define Function with Tunable Parameters

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        #units_tune = hp.Int("units", min_value=32, max_value=512, step=32)   #Use this if I only want 1 layer with tunable number of nodes
        dropout_tune = hp.Choice("dropout",[0.0,.1,.2,.5,.8])                 #Numbers must be float format 
        activation_tune = hp.Choice("activation", ["relu"])                   #To try multiple: hp.Choice("activation", ["relu", "linear"])
        l1_tune = 0.0                                   #lasso regularization
        l2_tune = hp.Choice("ridge", [0.01, 0.1,0.5,1.0,5.0,10.0,30.0])       #Numbers must be float format
        learning_rate_tune = hp.Float("learning_rate", min_value=1e-6, max_value=1e-1, sampling="log")   #Moves along the logarithmic curve  
        
        model = keras.Sequential()
        model.add(layers.Flatten())
        for i in range(hp.Int("num_layers", 1, 3)):
            model.add(
                layers.Dense(
                    units=hp.Int(f"units_{i}", min_value=32, max_value=256, step=32), #tune # of units and tune # of layers 
                    kernel_regularizer = regularizers.l1_l2(l1=l1_tune, l2=l2_tune),
                    activation=activation_tune,
                )
            )
        # Tune whether to use dropout.
        if hp.Boolean("dropout"):
            model.add(layers.Dropout(dropout_tune))
        model.add(layers.Dense(NLABEL, activation="softmax"))   #NLABEL is the number of output classes 

        model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate_tune),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
        
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [32, 64, 128, 256, 512]),
            **kwargs,
        )

tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="val_accuracy",     #use validation accuracy to choose which model is the best 
    max_trials=50,                #how many total trials to run during search 
    overwrite=True,
    directory=ddir_out,
    project_name= experiment_name,
)

In [29]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
dropout (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.1, 0.2, 0.5, 0.8], 'ordered': True}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu'], 'ordered': False}
ridge (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 30.0], 'ordered': True}
learning_rate (Float)
{'default': 1e-06, 'conditions': [], 'min_value': 1e-06, 'max_value': 0.1, 'step': None, 'sampling': 'log'}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}


In [30]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', #monitor='val_prediction_accuracy'
                                                   patience=25,
                                                   mode='auto',
                                                   restore_best_weights=True,
                                                   verbose=1)

In [31]:
tuner.search(X_train, onehotlabels, epochs=300, validation_data=(X_val, onehotlabels_val), callbacks = [es_callback, keras.callbacks.TensorBoard(ddir_out+"TensorBoard_output")])


Trial 50 Complete [00h 09m 59s]
val_accuracy: 0.5397939085960388

Best val_accuracy So Far: 0.5638657212257385
Total elapsed time: 11h 14m 59s
INFO:tensorflow:Oracle triggered exit


In [13]:
def get_best_model(tuner_results_dir):
    
    best_score = 0 #if score is based on val_loss, then change best_score = np.inf
    hps = dict()
    
    for dirnm in os.listdir(tuner_results_dir):
        if dirnm.startswith("trial"):
            with open(tuner_results_dir + '/' + dirnm + '/trial.json', 'r') as f:
                data = json.load(f)
                score = data['score']
                if score > best_score: #switch sign if score is based on val_loss 
                    best_score = score
                    hps = data['hyperparameters']['values']
                     
    return hps

In [14]:
bestmodel = get_best_model(ddir_out+experiment_name)
bestmodel

{'dropout': 0.0,
 'activation': 'relu',
 'ridge': 5.0,
 'learning_rate': 6.656054618657257e-06,
 'num_layers': 3,
 'units_0': 64,
 'units_1': 96,
 'batch_size': 512,
 'units_2': 96}